In [1]:
import numpy as np 
import os
import mne
import pandas as pd
from itertools import compress
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
import re
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

# import the data here (Sai)


In [7]:

#convert the data file from NIRs to CSV

# preprocess the data values (worked on by Thomas and Ian)
data_path = r'C:/Users/takoy/fnirs-github/neurotech_fnir_mind/data/'
for file in os.listdir(data_path):
    file_path = os.path.join(data_path, file)
    if ("nirs" in file or "csv" in file):
        continue
    raw_intensity = mne.io.read_raw_snirf(file_path, verbose=True)
    raw_intensity.load_data()

    picks = mne.pick_types(raw_intensity.info, meg=False, fnirs=True)
    dists = mne.preprocessing.nirs.source_detector_distances(
        raw_intensity.info, picks = picks
    )
    raw_intensity.pick(picks[dists > 0.01])
    
    raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)

    sci = mne.preprocessing.nirs.scalp_coupling_index(raw_od)

    raw_od.info['bads'] = list(compress(raw_od.ch_names, sci < 0.5))

    raw_haemo = mne.preprocessing.nirs.beer_lambert_law(raw_od, ppf=0.1)

    raw_haemo_unfiltered = raw_haemo.copy()
    raw_haemo.filter(0.05, 0.7, h_trans_bandwidth=0.2, l_trans_bandwidth=0.02)

    stim_ch = np.zeros((1, raw_haemo.n_times))
    manual_events = [5000, 10000, 15000]
    manual_codes = [1, 2, 3]

    for e, c in zip(manual_events, manual_codes):
        stim_ch[0, e] = c

    raw_haemo.add_channels([stim_ch], force_update_info=True)

    events = mne.find_events(raw_haemo, stim_channel=None, min_duration=0.01)
    event_dict = {1: "Control", 2: "Tapping (Left)", 3: "Tapping (Right)"}

    reject_criteria = dict(hbo=80e-6)
    tmin, tmax = -5, 15

    epochs = mne.Epochs(
        raw_haemo,
        events,
        event_id=event_dict,
        tmin=tmin,
        tmax=tmax,
        reject=reject_criteria,
        reject_by_annotation=True,
        proj=True,
        baseline=(None, 0),
        preload=True,
        detrend=None,
        verbose=True,
    )

    evoked_dict = {
        "Tapping/HbO": epochs["Tapping"].average(picks="hbo"),
        "Tapping/HbR": epochs["Tapping"].average(picks="hbr"),
        "Control/HbO": epochs["Control"].average(picks="hbo"),
        "Control/HbR": epochs["Control"].average(picks="hbr"),
    }

    # Rename channels until the encoding of frequency in ch_name is fixed
    for condition in evoked_dict:
        evoked_dict[condition].rename_channels(lambda x: x[:-4])

    color_dict = dict(HbO="#AA3377", HbR="b")
    styles_dict = dict(Control=dict(linestyle="dashed"))

    mne.viz.plot_compare_evokeds(
    evoked_dict, combine="mean", ci=0.95, colors=color_dict, styles=styles_dict
)



Loading C:\Users\takoy\fnirs-github\neurotech_fnir_mind\data\rest15.snirf
Reading 0 ... 22949  =      0.000 ...   458.980 secs...


C:\Users\takoy\AppData\Local\Temp\ipykernel_21720\3258200155.py:9: RuntimeWarning: The data only contains 2D location information for the optode positions. It is highly recommended that data is used which contains 3D location information for the optode positions. With only 2D locations it can not be guaranteed that MNE functions will behave correctly and produce accurate results. If it is not possible to include 3D positions in your data, please consider using the set_montage() function.
  raw_intensity = mne.io.read_raw_snirf(file_path, verbose=True)
C:\Users\takoy\AppData\Local\Temp\ipykernel_21720\3258200155.py:18: RuntimeWarning: Negative intensities encountered. Setting to abs(x)
  raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.05 - 0.7 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.05
- Lower transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.04 Hz)
- Upper passband edge: 0.70 Hz
- Upper transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.80 Hz)
- Filter length: 8251 samples (165.020 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.1s


AttributeError: 'numpy.ndarray' object has no attribute 'preload'

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = None

#X_train, X_test, y_train, y_test = train_test_split(, , test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions:
y_pred = model.predict(X_test)

#Get data from fNIRs (Elan)

# calculating the mean values.
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
cm = metrics.confusion_matrix(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)
plt.scatter(X_test, y_test, color='blue')
plt.plot(X_test, y_pred, color='red', linewidth=2)
plt.show()
